# Multi-Agent Architecture with Semantic Kernel and Azure API Management

## Workshop for RWS: Building a Proof of Concept

This workshop guides you through building a multi-agent architecture that leverages Semantic Kernel for orchestrating agents and Azure API Management (APIM) as an AI Gateway for function calling against backend services like Azure Functions and databases.

### Architecture Overview

```
┌─────────────────┐     ┌───────────────┐     ┌─────────────────┐     ┌────────────────┐
│                 │     │               │     │                 │     │                │
│  Semantic Kernel│     │  Azure API    │     │  Integration    │     │  Data Sources  │
│  Multi-Agents   │────▶│  Management   │────▶│  Services       │────▶│              │
│                 │     │  (AI Gateway) │     │  (Functions)    │     │                │
└─────────────────┘     └───────────────┘     └─────────────────┘     └────────────────┘
```

### What You'll Learn

1. How to set up Semantic Kernel with multiple specialized agents
2. How to connect agents to Azure API Management for function calling
3. How to orchestrate agent collaboration using different strategies
5. How to build specialized agents performing different tasks (RAG with AI Search, function calling, etc.)

### Implementation Approach

The implementation code is organized into Python modules in the `rws-app` directory for better maintainability and reuse. This notebook will focus on explaining concepts and demonstrating how to use these modules.

Let's begin.

## Step 1: Environment Setup and Prerequisites

First, let's install the necessary packages and set up our environment. We'll need Semantic Kernel and other supporting libraries.

In [17]:
# Install required packages
! pip install semantic-kernel==1.29.0 azure-identity python-dotenv requests pyodbc azure-storage-blob azure-search-documents

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/298.8 kB ? eta -:--:--
   -------- ------------------------------- 61.4/298.8 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 298.8/298.8 kB 4.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We'll use Python modules from the `rws-app` directory to organize our code. Let's import these modules:

In [12]:
import os
import sys

# Add the following directories to the Python path
sys.path.append(os.path.abspath("./rws-app"))
sys.path.append(os.path.abspath("../../../shared"))

# Import our utility functions
from utils import check_and_load_environment, display_environment_variables

required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_MODEL_DEPLOYMENT_NAME",
    "APIM_GATEWAY_URL",
    "APIM_SUBSCRIPTION_KEY",
]
# Make sure we have the necessary environment variables
check_and_load_environment(required_vars)

# Display the environment variables (masking sensitive ones)
display_environment_variables()

Environment Variables:
ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\alvanakievi\AppData\Roaming
APPLICATIONINSIGHTS_CONFIGURATION_CONTENT: {}
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL: 1
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\crashpad_25408_QYQTWXWSMCZNHTVK
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: EVI
COMSPEC: C:\Windows\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
EFC_8704: 1
ELECTRON_RUN_AS_NODE: 1
FPS_BROWSER_APP_PROFILE_STRING: Internet Explorer
FPS_BROWSER_USER_PROFILE_STRING: Default
GOPATH: C:\Users\alvanakievi\go
HOMEDRIVE: C:
HOMEPATH: \Users\alvanakievi
JPY_INTERRUPT_EVENT: 5520
LOCALAPPDATA: C:\Users\alvanakievi\AppData\Local
LOGONSERVER: \\EVI
NUMBER_OF_PROCESSORS: 8
ONEDRIVE: C:\Users\alvanakievi\OneDrive - Microsoft
ONEDRIVECOMMERCIAL: C:\Users\alvanakievi\OneDrive - Microsoft
ORIGINAL_XDG_CURRENT_DESKTOP: undefine

To use this workshop, you need to create a `.env` file with the following variables:

```
AZURE_OPENAI_ENDPOINT='[YOUR_ENDPOINT]'
AZURE_OPENAI_API_KEY='[YOUR_API_KEY]'
AZURE_OPENAI_MODEL_DEPLOYMENT_NAME='gpt-4o'
APIM_GATEWAY_URL='[YOUR_APIM_GATEWAY_URL]'
APIM_SUBSCRIPTION_KEY='[YOUR_SUBSCRIPTION_KEY]'
```

These will connect to your Azure resources, including Azure OpenAI and API Management.

## Step 2: Setting Up Semantic Kernel

Now, let's set up a Semantic Kernel instance that will be the foundation for our agent system. We'll use our `kernel_setup` module to create the kernel with the appropriate AI service.

In [ ]:
# Import the kernel setup function

# Create a main kernel for our agents


Kernel created successfully!


<details>
<summary>Click to see solution</summary>

```python

# Import the kernel setup function
from kernel_setup import create_kernel_with_service

# Create a main kernel for our agents
kernel = create_kernel_with_service(service_id="chat-completion")
print("Kernel created successfully!")

```
</details>

## Step 3: Adding Plugins

### Connecting to Azure API Management

Now, let's set up functions to call the API endpoints through APIM. These will be registered with our kernel and made available to the agents. 

The `ApiManagementPlugin` class in the `api_plugin.py` module provides:
- A connection to the API Management gateway
- Functions for getting weather data
- Functions for querying SQL databases
- Functions for retrieving sales data by region

In [ ]:
from api_plugin import ApiManagementPlugin

# Create an instance of the plugin
apim_plugin = ApiManagementPlugin()

# TODO Add the plugin to kernel


functions = kernel.get_plugin("ApiManagement").functions
print("RAG plugin registered with functions:")
print([f.name for f in functions.values()])

RAG plugin registered with functions:
['get_active_projects', 'get_asset_statistics', 'get_assets_by_region', 'get_critical_assets', 'get_safety_inspections']


<details>
<summary>Click to see solution</summary>

```python

# Add the plugin to kernel
kernel.add_plugin(apim_plugin, plugin_name="ApiManagement")

```
</details>

### Connecting to AI Search & Blob Storage

Next, let's connect our agents to Azure AI Search and Azure Blob Storage using the RAG (Retrieval Augmented Generation) plugin. The plugin can be found in the `RAGPlugin` class inside the `rag_plugin.py`. This plugin enables our agents to:

- Search for information in a knowledge base stored in Azure Blob Storage
- Use both keyword and semantic search capabilities of Azure AI Search
- Retrieve relevant documents with their metadata
- Enhance responses with specific domain knowledge

The RAG pattern improves agent responses by retrieving relevant domain-specific information before generating answers.

In [18]:
from rag_plugin import RAGPlugin

kernel.add_plugin(RAGPlugin(), plugin_name="RAGPlugin")

functions = kernel.get_plugin("RAGPlugin").functions
print("RAG plugin registered with functions:")
print([f.name for f in functions.values()])

RAG plugin registered with functions:
['search_knowledge_base']


Let's examine how the API plugin works. It leverages Semantic Kernel's function calling capabilities to allow LLMs to interact with external systems through API Management. This approach provides several benefits:

1. **Abstraction**: The agents don't need to know the details of how APIs are implemented
2. **Security**: API keys are managed securely through API Management
3. **Centralization**: All API calls go through a common gateway
4. **Monitoring**: API calls can be monitored and analyzed

Each function in the plugin is decorated with `@kernel_function` and includes parameter annotations to help the LLM understand how to use them.

## Step 4: Creating Specialized Agents

Now, let's create specialized agents that can perform different tasks using the functions we've registered. Each agent will have a specific role and expertise.

We have defined the following agents:
1. **Infrastructure Analys Agentt**: Analyzes asset conditions and safety
2. **Water Management Expert Agent**: Provides insights on water infrastructure
3. **Strategic Advisor Agent**: Provides long-term infrastructure recommendations
4. **Water Management Expert Agent**: Provides insights on water infrastructure
5. **Knowledge Agent**: Can access and retrieve information from the knowledge base
6. **Research Synthesis Agent**: Combines knowledge retrieval with infrastructure analysis

In [ ]:
# TODO Import our agent creation functions from agents.py

from semantic_kernel.connectors.ai import FunctionChoiceBehavior


# Create settings with auto function calling enabled
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id="chat-completion"
)


settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# TODO Create our specialized agents


# Store all agents in a list for convenience
agents = [
    infrastructure_analysis_agent,
    water_management_expert_agent,
    strategic_advisor_agent,
    knowledge_agent,
    research_synthesis_agent,
]

print(
    f"Created specialized agents: {', '.join([agent.name for agent in agents])}"
)

Created specialized agents: InfrastructureAnalyst, WaterManagementExpert, StrategicAdvisor, KnowledgeAgent, ResearchSynthesisAgent


<details>
<summary>Click to see solution</summary>

```python
# Import our agent creation functions from agents.py
from agents import (
    create_infrastructure_analyst_agent,
    create_water_management_expert_agent,
    create_strategic_advisor_agent,
    create_knowledge_agent,
    create_research_synthesis_agent,
)

# Create our specialized agents
infrastructure_analysis_agent = create_infrastructure_analyst_agent(kernel, settings)
water_management_expert_agent = create_water_management_expert_agent(kernel, settings)
strategic_advisor_agent = create_strategic_advisor_agent(kernel, settings)
knowledge_agent = create_knowledge_agent(kernel, settings)
research_synthesis_agent = create_research_synthesis_agent(kernel, settings)
```
</details>

Each agent is created with specific instructions that guide its behavior:

1. **DataAnalyst** focuses on retrieving and analyzing data from the SQL database
2. **EnvironmentalExpert** specializes in weather data and its implications
3. **BusinessAdvisor** synthesizes information and provides strategic recommendations

By creating specialized agents, we can benefit from:
- **Division of labor**: Each agent focuses on what it does best
- **Expertise**: Specialized prompts create more expert behavior
- **Clearer responsibilities**: Each agent has a defined role

## Step 5: Testing Individual Agents

Before creating a multi-agent system, let's test each agent individually to make sure they can perform their specialized tasks.
Here is a list of possible questions, designed to leverage on agents' expertise and access to external data.

Here are meaningful questions for each agent that leverage the SQL function endpoints:

For **Infrastructure Analysis Agent**:
1. "Can you analyze the critical infrastructure assets in Noord-Nederland and recommend prioritization for maintenance based on safety ratings and inspection findings?"
2. "What patterns do you see in safety inspection reports for bridges across different regions, and what preventive measures would you recommend?"
3. "Based on the active maintenance projects and safety ratings, which infrastructure types need the most immediate attention?"

For **Water Management Expert Agent**:
1. "Looking at safety inspection data for water-related infrastructure in coastal regions, what trends do you notice about maintenance needs?"
2. "How do safety ratings of water management assets compare across different regions, and what might explain these differences?"
3. "Based on recent inspection findings, what improvements would you recommend for water infrastructure maintenance protocols?"

For **Strategic Advisor Agent**:
1. "Using the asset statistics and safety inspection data, what strategic recommendations would you make for long-term infrastructure investment?"
2. "How should we adjust our maintenance strategy based on the distribution of critical assets across regions?"
3. "What policy recommendations would you make based on the correlation between inspection frequency and safety ratings?"

For **Knowledge Agent**:
1. "What were the key findings from the survey on cooperative systems for road operators?"
2. "What insights can you draw from comparing maintenance projects' priorities with their actual safety ratings?"
3. "What can you tell me about Asphalt paving at temperatures below freezing according to the Dutch Highways Authority?"

For **Research Synthesis Agent**:
1. "Can you synthesize the safety inspection findings and maintenance project data to identify emerging infrastructure challenges?"
2. "What research implications can you draw from the relationship between asset age and safety ratings across different infrastructure types?"
3. "Based on our inspection and maintenance data, what research areas should we prioritize for improving infrastructure resilience?"

In [20]:
# Import the test_agent function
from collaboration import test_agent

# Test the Data Analyst agent
await test_agent(infrastructure_analysis_agent, "Can you analyze the critical infrastructure assets in Noord-Nederland and recommend prioritization for maintenance based on safety ratings and inspection findings?")


=== Testing InfrastructureAnalyst ===

User: Can you analyze the critical infrastructure assets in Noord-Nederland and recommend prioritization for maintenance based on safety ratings and inspection findings?

InfrastructureAnalyst: I am currently unable to retrieve information regarding critical infrastructure assets, safety inspection reports, or maintenance projects due to access restrictions.

However, I can provide you with a general recommendation on how to prioritize maintenance for infrastructure assets:

1. **Safety Ratings**: Focus on assets with the lowest safety ratings. These pose immediate risks and should be addressed as a priority.

2. **Inspection Findings**: Review the most recent safety inspection reports for insights into specific issues. Pay attention to any urgent recommendations for repairs or reinforcements.

3. **Frequent Incidents**: Assets that have a history of frequent issues or incidents in the past should also be high on the maintenance list.

4. **Usage

AgentResponseItem(message=ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BTozetRV9HZzxfUvmDsFRnew4UaNN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am currently unable to retrieve information regarding critical infrastructure assets, safety inspection reports, or maintenance projects due to access restrictions.\n\nHowever, I can provide you with a general recommendation on how to prioritize maintenance for infrastructure assets:\n\n1. **Safety Ratings**: Focus on assets with the lowest safety ratings. These pose immediate risks and should be addressed as a priority.\n\n2. **Inspection Findings**: Review the most recent safety inspection reports for insights into specific issues. Pay attention to any urgent recommendations for repairs or reinforcements.\n\n3. **Frequent Incidents**: Assets that have a history of frequent issues or incidents in the past should also be high on the maintenance list.\n\n4. **Usage Lev

The **knowledge agent** is meant to search the knowledge base for relevant information. The knowledge base for this lab conists of 4 documents related to road maintanance found on the `https://open.rijkswaterstaat.nl/` website. 
- [Life-prolonging preventive maintenance techniques for porous asphalt](https://open.rijkswaterstaat.nl/zoeken/@55493/life-prolonging-preventive-maintenance/)
- [Innovatie Projecten Wegonderhoud](https://open.rijkswaterstaat.nl/zoeken/@215119/innovatieprojecten-wegonderhoud-ipw-road/#highlight=road%20maintenance)
- [Asphalt paving at temperatures below freezing](https://open.rijkswaterstaat.nl/zoeken/@205619/asphalt-paving-at-temperatures-below/#highlight=road%20maintenance)
- [Analysis for the road operators : results from a survey](https://open.rijkswaterstaat.nl/zoeken/@32834/analysis-for-the-road-operators-results/#highlight=road%20maintenance)

These files have been stored in a blob storage and indexed via AI Search. A local copy of these files can be found in the data folder. Try asking it quesitons about the contents of those files!

In [21]:
await test_agent(knowledge_agent, "What can you tell me about Asphalt paving at temperatures below freezing according to the Dutch Highways Authority?")


=== Testing KnowledgeAgent ===

User: What can you tell me about Asphalt paving at temperatures below freezing according to the Dutch Highways Authority?

KnowledgeAgent: According to the Dutch Highways Authority (RWS), the process of asphalt paving at temperatures below freezing involves several critical considerations and adjustments to ensure effective application and quality. Here are some key insights derived from a detailed report on this subject:

### Key Recommendations for Paving at Temperatures ≤ 0°C
1. **Temperature Requirements**:
   - The asphalt mixture should be maintained at a temperature of **160°C** to ensure sufficient compaction even in potential hold-ups during application.
   
2. **Binder Content Adjustments**:
   - For the winter mixture asphalt binder course (0/16 S), the binder content should be increased to at least **4.5% by mass** and **11.0% by volume**.
   - The stone mastic asphalt (0/8 S) should also see an increased binder content of **7.3% by mass** a

AgentResponseItem(message=ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BToztYHDS8fLxaWgDboKrfhUirYJt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='According to the Dutch Highways Authority (RWS), the process of asphalt paving at temperatures below freezing involves several critical considerations and adjustments to ensure effective application and quality. Here are some key insights derived from a detailed report on this subject:\n\n### Key Recommendations for Paving at Temperatures ≤ 0°C\n1. **Temperature Requirements**:\n   - The asphalt mixture should be maintained at a temperature of **160°C** to ensure sufficient compaction even in potential hold-ups during application.\n   \n2. **Binder Content Adjustments**:\n   - For the winter mixture asphalt binder course (0/16 S), the binder content should be increased to at least **4.5% by mass** and **11.0% by volume**.\n   - The stone mastic asphalt (0/8 S) should al

## Step 6: Setting Up Multi-Agent Collaboration - Sequential Approach

Now that we've tested each agent individually, let's create a multi-agent system where agents can collaborate to solve complex problems. We'll start with a simple sequential (round-robin) approach where agents take turns in a fixed order.

In [ ]:
# Import the collaboration functions
from collaboration import create_sequential_group, run_group_chat

# TODO First, create a sequential (round-robin) collaboration
 # 2 rounds of all 3 agents

print(f"Created a sequential group chat with {len(sequential_group.agents)} agents")
print(f"Maximum iterations: {sequential_group.termination_strategy.maximum_iterations}")

Created a sequential group chat with 5 agents
Maximum iterations: 6


<details>
<summary>Click to see solution</summary>

```python
sequential_group = create_sequential_group(agents, max_iterations=6) 
```
</details>

In [23]:
# Test the sequential group with a complex question
complex_query = "Given the current sales data and weather conditions in Europe, what strategic recommendations can you provide for our agricultural product sales in the next quarter?"

chat_history = await run_group_chat(sequential_group, complex_query)


User: Given the current sales data and weather conditions in Europe, what strategic recommendations can you provide for our agricultural product sales in the next quarter?

=== Beginning Agent Collaboration ===


## InfrastructureAnalyst:
I'm currently focused on analyzing infrastructure assets in the Netherlands, particularly regarding safety conditions and maintenance. For agricultural product sales and strategic recommendations based on sales data and weather conditions, I recommend consulting other experts or resources that specialize in agricultural economics, marketing strategies, or climate impact analysis. 

If you have any questions related to infrastructure, I'm here to help!

## WaterManagementExpert:
I'm currently focused on monitoring and analyzing water management and flood protection systems. For agricultural product sales recommendations, particularly in the context of weather conditions, it would be best to consult an expert in agricultural economics or marketing. 

I

### Understanding the Sequential Approach

In the sequential approach:

1. Agents take turns in the order they were added to the group
2. Each agent sees the full conversation history when generating its response
3. The conversation continues until it reaches the maximum number of iterations

This approach is simple and ensures each agent gets an equal opportunity to contribute. However, it may not be the most efficient for all tasks, as some tasks might benefit from a more structured workflow.

## Step 7: Creating a Custom Workflow for Specialized Collaboration

Now, let's create a more tailored collaboration pattern using a custom workflow. This allows us to define exactly which agent speaks in what order, creating a process that matches our business logic.

In [24]:
# Import the fixed workflow creation function
from collaboration import create_fixed_workflow_chat

# Define our specific workflow for agricultural decision-making:
# First gather data, then analyze environmental conditions, and finally provide business recommendations
agricultural_workflow = [
    "KnowledgeAgent", # First get data
    "WaterManagementExpert",  # Then check environmental conditions
    "StrategicAdvisor",       # Provide initial recommendations
    "KnowledgeAgent", # Do deeper data analysis based on recommendations
    "StrategicAdvisor"        # Final strategic recommendations
]

# Create the workflow chat
workflow_chat = create_fixed_workflow_chat(
    agents=[knowledge_agent, water_management_expert_agent, strategic_advisor_agent],
    workflow_sequence=agricultural_workflow,
    max_iterations=len(agricultural_workflow),
)

print(f"Created fixed workflow chat with sequence: {' → '.join(agricultural_workflow)}")


Created fixed workflow chat with sequence: KnowledgeAgent → WaterManagementExpert → StrategicAdvisor → KnowledgeAgent → StrategicAdvisor


In [25]:
# Test our custom workflow with a strategic business question
strategic_query = "How should we adapt our planting and distribution strategies for tomato seeds in Europe next season given current sales data and environmental trends?"

chat_history = await run_group_chat(workflow_chat, strategic_query)


User: How should we adapt our planting and distribution strategies for tomato seeds in Europe next season given current sales data and environmental trends?

=== Beginning Agent Collaboration ===

Selected: KnowledgeAgent

## KnowledgeAgent:
To adapt your planting and distribution strategies for tomato seeds in Europe for the upcoming season, it's essential to consider the current sales data and prevailing environmental trends. Here's a synthesized overview based on the latest insights:

### 1. **Sales Data Analysis**
   - **Top Performing Varieties**: Focus on cultivating and distributing tomato varieties that showed strong sales growth last season. This may include heirloom varieties or hybrids specifically suited for local climates.
   - **Regional Preferences**: Tailor your offerings based on regional preferences identified through sales data, ensuring that your distribution aligns with areas that recorded higher demand.

### 2. **Environmental Trends**
   - **Climate Adaptation**

### Understanding the Custom Workflow Approach

In the custom workflow approach:

1. We define a specific sequence of agents that mimics a business process
2. The workflow follows steps: data gathering → environmental analysis → initial recommendations → deeper analysis → final recommendations
3. Each agent still sees the full conversation history

This approach provides more control over the collaboration process and can lead to more predictable and structured outputs. It's particularly useful when you have a well-defined process that you want to follow.

## Conclusion

In this workshop, we've built a complete multi-agent system that leverages Semantic Kernel to create a powerful and flexible architecture.

We've covered:

1. Setting up a Semantic Kernel with Azure OpenAI
2. Creating API Management plugins for function calling
3. Designing specialized agents with different roles
4. Implementing different collaboration strategies between agents
5. Building custom workflows for domain-specific processes
6. Integrating Azure AI Agents for enhanced capabilities

The code is organized into maintainable modules in the `rws-app` directory, making it easy to extend and customize for your specific needs.

For further exploration, you can:
- Add more agents with different specializations
- Implement more complex workflow patterns
- Connect to additional APIs through API Management
- Expand the system with more sophisticated reasoning capabilities